In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
num_classes=5 # five emotions i.e neutral,happy,sad,angry,surprise
img_rows, img_cols=48,48
batch_size=32 # number of images per round

In [3]:
#getting the data
training_set='C:/Users/HP/Desktop/Emotion Dataset/fer2013/train'
testing_set='C:/Users/HP/Desktop/Emotion Dataset/fer2013/validation'

In [4]:
# generating train data
train_datagen=ImageDataGenerator(rescale=1./255,
                                rotation_range=30,
                                shear_range=0.3,
                                zoom_range=0.3,
                                width_shift_range=0.4,
                                height_shift_range=0.4,
                                horizontal_flip=True,
                                fill_mode='nearest')

train_generator=train_datagen.flow_from_directory(training_set,
                                                 color_mode='grayscale',
                                                 target_size=(img_rows,img_cols),
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 24256 images belonging to 5 classes.


In [5]:
# test data generation
test_datagen=ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_directory(testing_set,
                                                 color_mode='grayscale',
                                                 target_size=(img_rows,img_cols),
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 3006 images belonging to 5 classes.


In [6]:
# Block 1
model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [7]:
# Block-2 

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
# Block-3

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [9]:
# Block-4 

model.add(Conv2D(256,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
# Block-5
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [11]:
# Block=6
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [12]:
# Block=7
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [13]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 44, 44, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 32)       

In [14]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [15]:
#We will save the best model using model checkpoint
checkpoints =ModelCheckpoint('Emotion_Detection.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# we will stop the training when the validation loss will not improve
earlystop =EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=9,
                        verbose=1,
                        restore_best_weights=True)

# if model does not improve after certain epochs then it will reduce the learning rate.
reduce_lr=ReduceLROnPlateau(monitor='val_loss',
                           factor=0.2,
                           patience=3,
                           verbose=1,
                           min_delt=0.0001)

In [16]:
callbacks=[earlystop, checkpoints, reduce_lr]

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy',
             optimizer= Adam(lr=0.001),
             metrics=['accuracy'])

nb_train_samples= 24256
nb_validation_samples=3006
epochs=3

history=model.fit_generator(train_generator,
                           #steps_per_epoch=nb_train_samples//batch_size
                           steps_per_epoch=24256,
                           epochs=epochs,
                           #callbacks=callbacks,
                           validation_data=test_generator,
                           #validation_steps=nb_validation_samples//batch_size
                           validation_steps=3006)

Epoch 1/3
24256/24256 [==============================] - 13719s 566ms/step - loss: 1.3572 - accuracy: 0.4200 - val_loss: 1.1601 - val_accuracy: 0.5313
Epoch 2/3
24256/24256 [==============================] - 13537s 558ms/step - loss: 1.1616 - accuracy: 0.5312 - val_loss: 0.8828 - val_accuracy: 0.5968
Epoch 3/3
15203/24256 [=================>............] - ETA: 1:22:09 - loss: 1.1117 - accuracy: 0.5577